In [10]:
import json

from prettytable import PrettyTable
from stringMatcher import levenshtein_ratio_and_distance
from time import time

table = PrettyTable(field_names=["id", "fueltype", "carrating", "originalprice"])

# Load the database into the memory...

In [11]:
data = json.load(open("dataStore/dataFinal.json", "rb"))

# Load the Global Secondary index in the memory...

In [12]:
GIS = json.load(open("dataStore/dataFinal_GIS.json", "rb"))

# Write a function to search the cars by Car Name

In [13]:
def getClosestMatch(queryString):
    # Initialize the time counter...
    tic                  = time()
    
    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(queryString, KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the movieID of the movie which is the best match to the queryString...
    bestMatch_carName = list(GIS)[bestMatchRatio_index]
    bestMatch_CarID    = GIS[bestMatch_carName]

    # Now we have movieID which is the primary key in our main data-store...
    # Retrieve the details of the movie from the MovieID
    carInfo            = data[str(bestMatch_CarID)]
    
    # Mark the search completion time...
    toc                  = time()

    # Add Movie Title to the response...
    carInfo["car_name"] = bestMatch_carName
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : carInfo
    }
    
    return responseModel

# Write a function to make a request and visualize the data returned by the search function...

In [16]:
def request(carToSearch=""):
    # car name should always be in a string...
    carToSearch = str(carToSearch)
    # Check if the car name is valid...
    if(carToSearch.replace(" ", "").strip() == ""):
        return f"Invalid Car Name"
    
    # Make a request to fetch the movie info...
    response = getClosestMatch(queryString = carToSearch)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
   
    # Now we need to check if a user has given 
    data_to_print = f"\
    Carname          : {response['response body']['car_name']}\n\
    Fueltype          : {response['response body']['fueltype']}\n\
    Rating   : ---------------------------\n\
    "
    print(data_to_print)
    print(table)

In [17]:
request(carToSearch="hyundai")

    Carname          : hyundai eon
    Fueltype          : ['petrol']
    Rating   : ---------------------------
    
+----+----------+-----------+---------------+
| id | fueltype | carrating | originalprice |
+----+----------+-----------+---------------+
+----+----------+-----------+---------------+
